In [1]:
pip install pandas pyarrow

Note: you may need to restart the kernel to use updated packages.


In [2]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 차원 축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth

# 학습 모델 저장을 위한 라이브러리
import pickle

### 신용정보

In [3]:
# 신용정보 7월부터 12월까지 데이터를 불러오기
train_df07 = pd.read_parquet('open/train/2.신용정보/201807_train_신용정보.parquet')
train_df08 = pd.read_parquet('open/train/2.신용정보/201808_train_신용정보.parquet')
train_df09 = pd.read_parquet('open/train/2.신용정보/201809_train_신용정보.parquet')
train_df10 = pd.read_parquet('open/train/2.신용정보/201810_train_신용정보.parquet')
train_df11 = pd.read_parquet('open/train/2.신용정보/201811_train_신용정보.parquet')
train_df12 = pd.read_parquet('open/train/2.신용정보/201812_train_신용정보.parquet')

In [4]:
# 데이터 병합하기
# ignore_index = True : 인덱스를 새로 0부터 재부여한다.
train_df = pd.concat([train_df07, train_df08, train_df09, train_df10, train_df11, train_df12], ignore_index = True)
# 처음과 마지막 5개 행 출력해서 확인하기
# print(train_df.head())
# print(train_df.tail())

In [6]:
# segment 데이터 불러오기
segment = pd.read_parquet('open/train/1.회원정보/201807_train_회원정보.parquet')
segment_col = segment[['ID', 'Segment']]

In [8]:
# segment left outer join
train_df = train_df.merge(segment_col, on='ID', how='left')
train_df.head()

,기준년월,ID,최초한도금액,카드이용한도금액,CA한도금액,일시상환론한도금액,월상환론한도금액,CA이자율_할인전,CL이자율_할인전,RV일시불이자율_할인전,...,한도심사요청건수,한도요청거절건수,한도심사요청후경과월,한도심사거절후경과월,시장단기연체여부_R6M,시장단기연체여부_R3M,시장연체상환여부_R6M,시장연체상환여부_R3M,rv최초시작후경과일,Segment
0,201807,TRAIN_000000,0,19354,7270,0,0,22.995207,18.254978,17.264967,...,0회,0,3,3,0,0,0,0,99999999,D
1,201807,TRAIN_000001,0,9996,5718,41996,90611,14.793821,14.834873,10.622446,...,0회,0,3,3,0,0,0,0,322,E
2,201807,TRAIN_000002,0,88193,35207,0,0,22.014276,17.875321,17.155829,...,0회,0,3,3,0,0,0,0,2378,C
3,201807,TRAIN_000003,0,19062,6531,0,0,22.998014,22.999453,19.293674,...,0회,0,3,3,0,0,0,0,99999999,D
4,201807,TRAIN_000004,0,177222,47149,48000,155020,14.661948,10.897410,10.654587,...,0회,0,3,3,0,0,0,0,99999999,E


In [9]:
# 각 컬럼의 데이터 타입 확인하기
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 43 columns):
 #   Column          Dtype  
---  ------          -----  
 0   기준년월            int64  
 1   ID              object 
 2   최초한도금액          int64  
 3   카드이용한도금액        int64  
 4   CA한도금액          int64  
 5   일시상환론한도금액       int64  
 6   월상환론한도금액        int64  
 7   CA이자율_할인전       float64
 8   CL이자율_할인전       float64
 9   RV일시불이자율_할인전    float64
 10  RV현금서비스이자율_할인전  float64
 11  RV신청일자          float64
 12  RV약정청구율         float64
 13  RV최소결제비율        float64
 14  자발한도감액횟수_R12M   object 
 15  자발한도감액금액_R12M   int64  
 16  자발한도감액후경과월      int64  
 17  강제한도감액횟수_R12M   int64  
 18  강제한도감액금액_R12M   int64  
 19  강제한도감액후경과월      int64  
 20  한도증액횟수_R12M     object 
 21  한도증액금액_R12M     int64  
 22  한도증액후경과월        int64  
 23  상향가능한도금액        int64  
 24  상향가능CA한도금액      int64  
 25  카드론동의여부         object 
 26  월상환론상향가능한도금액    int64  
 27  RV전환가능여부        object 
 28  일시불ONLY전환가능여

In [11]:
# 결측치 확인
print(train_df.isnull().sum())
# RV신청일, RV 전환 가능여부에서 결측치 발견
# 상관계수 측정 후 0.5가 넘는 컬럼에 대해서 해당 컬럼들이 있을시 제거

기준년월                    0
ID                      0
최초한도금액                  0
카드이용한도금액                0
CA한도금액                  0
일시상환론한도금액               0
월상환론한도금액                0
CA이자율_할인전               0
CL이자율_할인전               0
RV일시불이자율_할인전            0
RV현금서비스이자율_할인전          0
RV신청일자            1951236
RV약정청구율                 0
RV최소결제비율                0
자발한도감액횟수_R12M           0
자발한도감액금액_R12M           0
자발한도감액후경과월              0
강제한도감액횟수_R12M           0
강제한도감액금액_R12M           0
강제한도감액후경과월              0
한도증액횟수_R12M             0
한도증액금액_R12M             0
한도증액후경과월                0
상향가능한도금액                0
상향가능CA한도금액              0
카드론동의여부                 0
월상환론상향가능한도금액            0
RV전환가능여부            29473
일시불ONLY전환가능여부           0
카드이용한도금액_B1M            0
카드이용한도금액_B2M            0
특별한도보유여부_R3M            0
연체감액여부_R3M              0
한도심사요청건수                0
한도요청거절건수                0
한도심사요청후경과월              0
한도심사거절후경과월              0
시장단기연체여부_R6M            0
시장단기연체여부_R3M

In [13]:
df1 = train_df.copy()

# Segment 라벨 인코딩
le_segment = LabelEncoder()
df1['Segment_enc'] = le_segment.fit_transform(df1['Segment'])

# 숫자형 컬럼만 선택
numeric_cols = df1.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Segment_enc가 없으면 추가
if 'Segment_enc' not in numeric_cols:
    numeric_cols.append('Segment_enc')

# 숫자형 컬럼으로 상관계수 계산
corr = df1[numeric_cols].corr()

# Segment_enc와 다른 변수들 상관계수 출력
segment_corr = corr['Segment_enc'].sort_values(ascending=False)
print(segment_corr)

# 소수점 4자리 고정 출력 설정
pd.set_option('display.float_format', '{:.2f}'.format)

print(segment_corr)

Segment_enc       1.000000e+00
rv최초시작후경과일        7.015499e-02
RV신청일자            5.777155e-02
한도증액후경과월          3.939048e-02
강제한도감액금액_R12M     3.743784e-02
일시불ONLY전환가능여부     1.920487e-02
월상환론상향가능한도금액      1.138778e-02
강제한도감액횟수_R12M     3.853306e-03
특별한도보유여부_R3M      2.992227e-03
자발한도감액후경과월        2.105123e-03
한도심사거절후경과월        1.723674e-03
시장단기연체여부_R6M      1.307753e-03
한도심사요청후경과월        1.211940e-03
시장단기연체여부_R3M      1.071772e-03
시장연체상환여부_R6M      2.984966e-04
기준년월              9.208416e-14
한도요청거절건수         -1.737019e-03
자발한도감액금액_R12M    -6.140816e-03
연체감액여부_R3M       -1.367032e-02
RV최소결제비율         -3.168042e-02
CL이자율_할인전        -3.726602e-02
강제한도감액후경과월       -4.460055e-02
월상환론한도금액         -5.027162e-02
상향가능한도금액         -5.076396e-02
RV약정청구율          -5.457546e-02
상향가능CA한도금액       -6.457483e-02
RV일시불이자율_할인전     -7.383820e-02
한도증액금액_R12M      -8.418015e-02
일시상환론한도금액        -9.419669e-02
RV현금서비스이자율_할인전   -9.687040e-02
최초한도금액           -1.070688e-01
CA이자율_할인전        -1.181894e-01
CA한도금액  

In [ ]:
# 상관계수 결과를 딕셔너리 형태로 작성 (예시)
corr_dict = {
    '기준년월': 0.00,
    'ID': np.nan,
    '대표결제일': 0.00,
    '대표결제방법코드': np.nan,
    '대표청구지고객주소구분코드': np.nan,
    '대표청구서수령지구분코드': np.nan,
    '청구서수령방법': np.nan,
    '청구서발송여부_B0': -0.21,
    '청구서발송여부_R3M': -0.18,
    '청구서발송여부_R6M': -0.15,
    '청구금액_B0': -0.58,
    '청구금액_R3M': -0.59,
    '청구금액_R6M': -0.60,
    '포인트_마일리지_건별_B0M': -0.22,
    '포인트_마일리지_건별_R3M': -0.22,
    '포인트_포인트_건별_B0M': -0.14,
    '포인트_포인트_건별_R3M': -0.15,
    '포인트_마일리지_월적립_B0M': -0.18,
    '포인트_마일리지_월적립_R3M': -0.18,
    '포인트_포인트_월적립_B0M': -0.12,
    '포인트_포인트_월적립_R3M': -0.13,
    '포인트_적립포인트_R12M': -0.18,
    '포인트_적립포인트_R3M': -0.17,
    '포인트_이용포인트_R12M': -0.14,
    '포인트_이용포인트_R3M': -0.17,
    '포인트_잔여포인트_B0M': 0.00,
    '마일_적립포인트_R12M': -0.21,
    '마일_적립포인트_R3M': -0.28,
    '마일_이용포인트_R12M': -0.21,
    '마일_이용포인트_R3M': -0.21,
    '마일_잔여포인트_B0M': -0.18,
    '할인건수_R3M': np.nan,
    '할인금액_R3M': -0.19,
    '할인건수_B0M': np.nan,
    '할인금액_B0M': -0.18,
    '할인금액_청구서_R3M': -0.20,
    '할인금액_청구서_B0M': -0.20,
    '상환개월수_결제일_R6M': -0.15,
    '상환개월수_결제일_R3M': -0.13,
    '선결제건수_R6M': -0.16,
    '선결제건수_R3M': -0.15,
    '연체건수_R6M': -0.06,
    '연체건수_R3M': -0.06,
    '혜택수혜금액_R3M': -0.36,
    '포인트_마일리지_환산_B0M': -0.28,
    '혜택수혜금액': -0.35,
}

# DataFrame 생성
corr_df = pd.DataFrame(list(corr_dict.items()), columns=['변수명', 'Segment_enc_상관계수'])

# 엑셀 파일로 저장
corr_df.to_excel('open.xlsx', index=False)